# event tables: dumps to sql

## intro

### imports

In [5]:
import os, pandas as pd, numpy as np
import pyperclip
from datetime import datetime as dt

import mysql.connector as mysql
import sqlalchemy

In [2]:
datapath = '../data/raw/jawiki/dumps_unzipped/'

### head, metadata

### notes

References:
- load data infile ([link](https://mariadb.com/kb/en/load-data-infile/))  
- fix an error stopping load data infile ([link 2](https://stackoverflow.com/questions/18437689/error-1148-the-used-command-is-not-allowed-with-this-mysql-version))
- set case ([link](https://stackoverflow.com/questions/58626827/how-can-i-set-mysql-loaded-table-items-in-a-column-using-a-case-when-statement))
- straightforward example ([link](https://stackoverflow.com/questions/12530971/mysql-multiple-set-on-load-data-infile))

### EDA and learning proccess

split file so I can run csvsql to generate schema for debugging (inactive)

#### copy 500 rows of the tsv

#### confirm the rows were copied

#### hand-edited query (1st draft) CREATE TABLE

##### before editing

#### hand-edited query (2nd draft) CREATE TABLE

#### simpler csv for debugging

In [ ]:
# pd.read_csv('../data/processed/jawiki_2013_simple.tsv', sep='\t')

## create table ja_hist

#### login

```bash
mysql --user=root --password=XXXXXXXX
```

```sql
CREATE DATABASE ja_hist CHARACTER SET utf8 COLLATE utf8_bin;
USE ja_wiki;
GRANT ALL PRIVILEGES ON ja_hist TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
GRANT ALL PRIVILEGES ON ja_hist.* TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
EXIT
```

#### edit mariadb.cnf

peek at ```/etc/mysql/mariadb.cnf```   
utf8 stuff (uncomment the lines)

#### passwords

In [ ]:
# mysql_superuser = 'root'
# mysql_su_pass = input(f'Enter the MySQL password for user {mysql_superuser}: ')
mysql_user = 'bhrdwj'
# mysql_pass = input(f'Enter the MySQL password for user {mysql_user}: ')

#### connect to database

In [ ]:
host='localhost'; user=mysql_user; passwd=mysql_pass; dbname='ja_hist';
cxn = mysql.connect(host=host,user=user,passwd=passwd, database=dbname)
cur = cxn.cursor()

connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
try:
    engine1 = sqlalchemy.create_engine(connection_str)
    conn1 = engine1.connect()
except:
    print('Database connection error - check creds')
metadata = sqlalchemy.MetaData(conn1)
metadata.reflect()
metadata.tables.keys()

#### prep for create table

In [ ]:
make_history_table = """
CREATE TABLE ja_hist ( 
wiki_db VARCHAR(20) NOT NULL,
event_entity VARCHAR(20) NOT NULL,
event_type VARCHAR(20) NOT NULL,
event_timestamp VARCHAR(30) NOT NULL,
event_comment TEXT,
event_user_id BIGINT,
event_user_text_historical TEXT,
event_user_text TEXT,
event_user_blocks_historical VARCHAR(30),
event_user_blocks VARCHAR(60),
event_user_groups_historical TEXT,
event_user_groups TEXT,
event_user_is_bot_by_historical VARCHAR(10),
event_user_is_bot_by VARCHAR(10),
event_user_is_created_by_self VARCHAR(10), # BOOL,
event_user_is_created_by_system VARCHAR(10), # BOOL,
event_user_is_created_by_peer VARCHAR(10), # BOOL,
event_user_is_anonymous VARCHAR(10), # BOOL,
event_user_registration_timestamp VARCHAR(30),
event_user_creation_timestamp VARCHAR(30),
event_user_first_edit_timestamp VARCHAR(30),
event_user_revision_count BIGINT,
event_user_seconds_since_previous_revision BIGINT,
page_id BIGINT,
page_title_historical VARCHAR(200),
page_title VARCHAR(200),
page_namespace_historical INT,
page_namespace_is_content_historical VARCHAR(10), # BOOL,
page_namespace INT,
page_namespace_is_content VARCHAR(10), # BOOL,
page_is_redirect VARCHAR(20), # BOOL,
page_is_deleted VARCHAR(20), # BOOL,
page_creation_timestamp VARCHAR(30),
page_first_edit_timestamp VARCHAR(30),
page_revision_count BIGINT,
page_seconds_since_previous_revision BIGINT,
user_id BIGINT,
user_text_historical VARCHAR(100),
user_text VARCHAR(100),
user_blocks_historical VARCHAR(50),
user_blocks VARCHAR(50),
user_groups_historical VARCHAR(50),
user_groups VARCHAR(50),
user_is_bot_by_historical VARCHAR(50),
user_is_bot_by VARCHAR(50),
user_is_created_by_self VARCHAR(50), # BOOL,
user_is_created_by_system VARCHAR(50), # BOOL,
user_is_created_by_peer VARCHAR(50), # BOOL,
user_is_anonymous VARCHAR(10), # BOOL,
user_registration_timestamp VARCHAR(30),
user_creation_timestamp VARCHAR(30),
user_first_edit_timestamp VARCHAR(30),
revision_id BIGINT,
revision_parent_id BIGINT,
revision_minor_edit VARCHAR(10), # BOOL,
revision_deleted_parts VARCHAR(30),
revision_deleted_parts_are_suppressed VARCHAR(10), # BOOL,
revision_text_bytes BIGINT,
revision_text_bytes_diff BIGINT,
revision_text_sha1 VARCHAR(50),
revision_content_model VARCHAR(10), # BOOL,
revision_content_format VARCHAR(10), # BOOL,
revision_is_deleted_by_page_deletion VARCHAR(10), # BOOL,
revision_deleted_by_page_deletion_timestamp VARCHAR(30),
revision_is_identity_reverted VARCHAR(10), # BOOL,
revision_first_identity_reverting_revision_id BIGINT,
revision_seconds_to_identity_revert BIGINT,
revision_is_identity_revert VARCHAR(10), # BOOL,
revision_is_from_before_page_creation VARCHAR(10), # BOOL,
revision_tags VARCHAR(50),
dumpfile_row_id BIGINT NOT NULL AUTO_INCREMENT,
row_id BIGINT NOT NULL AUTO_INCREMENT,

PRIMARY KEY (row_id)
);
"""

### insert data

In [ ]:
colnames_str = 'wiki_db,event_entity,event_type,event_timestamp,event_comment,event_user_id,event_user_text_historical,event_user_text,event_user_blocks_historical,event_user_blocks,event_user_groups_historical,event_user_groups,event_user_is_bot_by_historical,event_user_is_bot_by,event_user_is_created_by_self,event_user_is_created_by_system,event_user_is_created_by_peer,event_user_is_anonymous,event_user_registration_timestamp,event_user_creation_timestamp,event_user_first_edit_timestamp,event_user_revision_count,event_user_seconds_since_previous_revision,page_id,page_title_historical,page_title,page_namespace_historical,page_namespace_is_content_historical,page_namespace,page_namespace_is_content,page_is_redirect,page_is_deleted,page_creation_timestamp,page_first_edit_timestamp,page_revision_count,page_seconds_since_previous_revision,user_id,user_text_historical,user_text,user_blocks_historical,user_blocks,user_groups_historical,user_groups,user_is_bot_by_historical,user_is_bot_by,user_is_created_by_self,user_is_created_by_system,user_is_created_by_peer,user_is_anonymous,user_registration_timestamp,user_creation_timestamp,user_first_edit_timestamp,revision_id,revision_parent_id,revision_minor_edit,revision_deleted_parts,revision_deleted_parts_are_suppressed,revision_text_bytes,revision_text_bytes_diff,revision_text_sha1,revision_content_model,revision_content_format,revision_is_deleted_by_page_deletion,revision_deleted_by_page_deletion_timestamp,revision_is_identity_reverted,revision_first_identity_reverting_revision_id,revision_seconds_to_identity_revert,revision_is_identity_revert,revision_is_from_before_page_creation,revision_tags,row_id'
colnames = colnames_str.split(',')

coltypes_str = 'STR,STR,STR,STR,STR,INT,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,INT,INT,INT,STR,STR,INT,STR,INT,STR,STR,STR,STR,STR,INT,INT,INT,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,STR,INT,INT,STR,STR,STR,INT,INT,STR,STR,STR,STR,STR,STR,INT,INT,STR,STR,STR,INT'
coltypes = coltypes_str.split(',')

varnames_str = '@' + ',@'.join(map(str,colnames))
varnames = varnames_str.split(',')

# SET four = NULLIF(@vfour,'')
set_strcols_list = [f"{col} = NULLIF(@{col},'')" for col in colnames]
# SET contributor_zipcode = IF(@contributor_zipcode='',0,@contributor_zipcode)
# set_intcols_list = [f"{col} = IF(@{col}='',0,@{col})" for col in colnames]
# set_cols_list = [      set_strcols_list[i] 
                 # if    coltypes[i]=='STR' 
                 # else  set_intcols_list[i]
                 # for i in range(len(colnames))
                  # ]
set_cols_list = set_strcols_list

set_cols_str = ',\n'.join(set_cols_list)

basepath = '/home/bhrdwj/git/predwikt/data/raw/jawiki/dumps_unzipped/2021-12.jawiki.'
years = list(map(str,range(2013,2022)))
tsv_filepaths = [basepath+year+'.tsv' for year in years]

In [ ]:
load_data_sql = [
r"""
LOAD DATA LOCAL INFILE '""" +tsv_filepath+ r"""'
INTO TABLE ja_hist 
CHARACTER SET utf8mb4
FIELDS TERMINATED BY '\t'
LINES TERMINATED BY '\n'
(""" +varnames_str+ r""")
SET
""" +set_cols_str+ r"""
;
"""
    for tsv_filepath in tsv_filepaths
]
# this is a list comprehension that makes a list of queries, one for each tsv file

#### EDA for data cleaning

```sql
select revision_minor_edit, min(row_id), count(*) from ja_hist group by revision_minor_edit order by row_id;
select user_is_anonymous, min(row_id), count(*) from ja_hist group by user_is_anonymous order by row_id;
select user_text_historical, min(row_id), count(*) from ja_hist where user_text_historical regexp '^[0-9]*$' group by user_text_historical order by row_id;
select revision_content_model, min(row_id), count(*) from ja_hist group by revision_content_model order by row_id;
select user_is_bot_by_historical, min(row_id), count(*) from ja_hist group by user_is_bot_by_historical order by row_id;
select row_id, page_id, page_title, page_namespace, page_revision_count from ja_hist order by page_revision_count desc limit 30;


```


##### copy stuff to clipboard

### Make ja_hist

#### load the database

In [ ]:
from IPython.display import clear_output

In [ ]:
for load_query in [load_data_sql[0]]:
    a = conn1.execute(load_query)
    

#### something

In [ ]:
conn1.execute('DROP TABLE IF EXISTS ja_hist;')
conn1.execute(make_history_table)

In [ ]:
a = conn1.execute('describe ja_hist;')
a.all();

#### python

In [ ]:
a = conn1.execute('drop table if exists ja_hist;')
a.fetchall()

In [ ]:
conn1.execute(load_data_sql)

START 19:24PM

In [ ]:
print(dt.now())
print(dt.now())

In [ ]:
a = conn1.execute('select * from ja_hist;')
a.fetchall()